In [55]:
import dotenv, os
from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase
import pandas as pd

In [56]:
os.chdir("/Users/zokum/local-Workspace/misc/degen_dash")
# load .env file
dotenv.load_dotenv(".env")
# setup Dune Python client
dune = DuneClient.from_env()

In [71]:
# query = QueryBase(
#     query_id=3490743,
# #     params=[
# #         QueryParameter.text_type(name="TextField", value="Word"),
# #         QueryParameter.number_type(name="NumberField", value=3.1415926535),
# #         QueryParameter.date_type(name="DateField", value="2022-05-04 00:00:00"),
# #         QueryParameter.enum_type(name="EnumField", value="Option 1"),
# #     ],
# )
# 
# query_result = dune.run_query_dataframe(
#     query = query,
#     ping_frequency = 10,
#     performance = 'large'
# )

# query_result = dune.get_latest_result_dataframe(3490743)
query_result = pd.DataFrame.from_dict(dune.get_latest_result(3490743).result.rows)

In [72]:
query_result

,address,degen_amount,evt_tx_hash,minute,price_at_time
0,0x27d507b40f0ae4011f689e708b174e925c89cc69,-1.000000e+04,0x964dcef0e92943733e10fb86535f14aac8dd5ec60089...,2024-02-01 02:32:00.000 UTC,8.520000e-04
1,0x27d507b40f0ae4011f689e708b174e925c89cc69,-1.278587e+03,0x964dcef0e92943733e10fb86535f14aac8dd5ec60089...,2024-02-01 02:32:00.000 UTC,8.520000e-04
2,0x50e388c498ad8a92e57066b69108e2668b2a4998,-3.000000e+03,0x737da2d687b2c0ba1838a6faa903fd14ec06e11afbbb...,2024-02-01 02:32:00.000 UTC,8.520000e-04
3,0xdc74846f63d13669e7b2c61f7a18a0288529779f,-6.256487e+04,0xbc24eb338778f2bde6a3cadc973cfbca6ce4f04664f0...,2024-02-01 00:24:00.000 UTC,8.070000e-04
4,0xcb5d4d5a89c801ddf85f030709d115436d325d8c,-5.468346e+04,0x2a72c2dacca0f31a594dd66e8a5954e50b9149b73439...,2024-02-01 00:24:00.000 UTC,8.070000e-04
...,...,...,...,...,...
519599,0x1007a39088c22a4dfe54032f08fc47a7303603df,1.624925e+06,0x681076530c436ab398c68131c03646d1f110cd2ff389...,2024-01-09 09:15:00.000 UTC,9.703826e-07
519600,0x6207f31163e6f31d588019afaa14299a418d3264,5.663361e+06,0xd1be4a81f6c2e66dc5dd1895e366de807066bc7617b3...,2024-01-09 18:25:00.000 UTC,2.004702e-05
519601,0xc9034c3e7f58003e6ae0c8438e7c8f4598d5acaa,9.500000e+07,0xdf89ebffbb142c74b99cceeb408511cbfd91400718ab...,2024-01-11 16:09:00.000 UTC,9.703826e-07
519602,0xc9034c3e7f58003e6ae0c8438e7c8f4598d5acaa,2.537970e+05,0xa0b311561f31da8bd0269a9799fe123cccaeda9bd5b3...,2024-01-12 04:41:00.000 UTC,9.703826e-07


In [73]:
df = query_result

In [16]:
# for testing
# df = df[df['address'] == '0x58fec77fcd151f4aa0ee2a0c4a576111f1049d1d']

In [74]:
# Convert 'minute' to datetime and sort the dataframe
df['minute'] = pd.to_datetime(df['minute'])
df.sort_values(by=['address', 'minute'], inplace=True)

In [75]:
%%time

def calculate_realized_balance(group):
    virtual_coins = []  # This will hold tuples of (amount, price)

    # First pass: construct the list of virtual coins, ordered by amount (descending)
    for _, transaction in group.iterrows():
        amount = transaction['degen_amount']
        price = transaction['price_at_time']

        if amount > 0:
            # Incoming payment, add a new virtual coin
            virtual_coins.append((amount, price))
        else:
            # Outgoing payment, need to subtract from the largest available coin
            amount = -amount  # Make the amount positive
            virtual_coins = sorted(virtual_coins, key=lambda x: -x[0])  # Sort by amount descending

            for i, (virtual_coin_amount, virtual_coin_price) in enumerate(virtual_coins):
                if amount >= virtual_coin_amount:
                    # Use up the entire virtual coin
                    amount -= virtual_coin_amount
                    virtual_coins[i] = (0, price)  # This coin is now used up
                else:
                    # Only part of the virtual coin is used
                    virtual_coins[i] = (virtual_coin_amount - amount, price)
                    break  # We've subtracted the outgoing amount, so break the loop

        # Remove any coins that have been used up (amount is zero)
        virtual_coins = [coin for coin in virtual_coins if coin[0] > 0]
        
#         print("current virtual_coins = ")
#         print(virtual_coins)
#         print()

    # Second pass: calculate the realized balance from the remaining virtual coins
    realized_balance = sum(amount * price for amount, price in virtual_coins)
    
    return realized_balance

In [76]:
# Calculate realized balance for each address
df_grouped = df.groupby('address').apply(calculate_realized_balance)

/var/folders/d7/px4831yx38d7k85rs8_2n93r0000gn/T/ipykernel_95889/432536209.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df.groupby('address').apply(calculate_realized_balance)


In [78]:
# Summarize the realized balances to get the total realized cap
realized_cap = df_grouped.sum()
realized_cap, df_grouped

(78591014.41494605,
 address
 0x0000000000000000000000000000000000000000       167.777342
 0x0000000000002bdbf1bf3279983603ec279cc6df         0.121411
 0x00000000000ba9cd9f5175108141a82b6c24d727      8562.705691
 0x00000000009726632680fb29d3f7a9734e3010e2    237167.082520
 0x0000000000a6f0986c92cf1ec4d2e77afbe1466d     46429.138728
                                                   ...      
 0xfff0c2b705be5fb8245246550bf0a4dd233a70c0         1.044900
 0xfff9619cf4d7f254cff6d6e2aeea100591a80a74        48.385000
 0xfffb40347816873ca038756cc87bc610144166b2       116.564629
 0xfffe980de8f812bf3731d8f0682e21a2177bb40a         0.000000
 0xffffa219a6cd248ed05f13af39b24715db85e4e2         0.030972
 Length: 36735, dtype: float64)

In [80]:
def calculate_realized_balance_with_timestamp_and_return_df(group):
    virtual_coins = []  # This will hold tuples of (amount, price)
    data_for_df = [] 

    for index, transaction in group.iterrows():
        address = transaction['address']
        timestamp = transaction['minute']
        amount = transaction['degen_amount']
        price = transaction['price_at_time']

        if amount > 0:
            # Incoming payment, add a new virtual coin
            virtual_coins.append((amount, price))
        else:
            # Outgoing payment, need to subtract from the largest available coin
            amount = -amount # Make the amount positive  
            virtual_coins.sort(key=lambda x: -x[0]) # Sort by amount descending

            for i, (virtual_coin_amount, virtual_coin_price) in enumerate(virtual_coins):
                if amount >= virtual_coin_amount:
                    # Use up the entire virtual coin
                    amount -= virtual_coin_amount 
                    virtual_coins[i] = (0, price) # This coin is now used up
                else:
                    # Only part of the virtual coin is used
                    virtual_coins[i] = (virtual_coin_amount - amount, price)
                    amount = 0
                    break  # We've subtracted the outgoing amount, so break the loop
            
            # Remove any coins that have been used up (amount is zero)
            virtual_coins = [coin for coin in virtual_coins if coin[0] > 0]

        realized_balance = sum(coin_amount * coin_price for coin_amount, coin_price in virtual_coins)
        data_for_df.append({'address': address, 'timestamp': timestamp, 'realized_balance': realized_balance})

    
    return pd.DataFrame(data_for_df)

In [81]:
df = query_result

In [82]:
# Using the mocked DataFrame to demonstrate this function
# Let's create a new mock dataframe with timestamps for demonstration
df['minute'] = pd.to_datetime(df['minute'])  # Ensure 'minute' is a datetime column
grouped = df.groupby('address')  # Group by address

In [83]:
%%time

# Now let's apply the modified function to each group
result_dfs = [calculate_realized_balance_with_timestamp_and_return_df(group) for _, group in grouped]

CPU times: user 3min 37s, sys: 1.31 s, total: 3min 38s
Wall time: 3min 42s


In [84]:
# Concatenate all the individual DataFrames into a single one
final_df = pd.concat(result_dfs, ignore_index=True)
final_df

,address,timestamp,realized_balance
0,0x0000000000000000000000000000000000000000,2024-01-07 15:25:00+00:00,0.000000
1,0x0000000000000000000000000000000000000000,2024-01-08 17:49:00+00:00,0.000425
2,0x0000000000000000000000000000000000000000,2024-01-25 15:18:00+00:00,0.000425
3,0x0000000000000000000000000000000000000000,2024-01-25 18:02:00+00:00,0.076445
4,0x0000000000000000000000000000000000000000,2024-02-27 14:08:00+00:00,167.777342
...,...,...,...
519599,0xfffb40347816873ca038756cc87bc610144166b2,2024-02-04 10:25:00+00:00,116.564629
519600,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-28 05:20:00+00:00,9577.678563
519601,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-29 02:17:00+00:00,10157.923101
519602,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-29 02:17:00+00:00,0.000000


In [85]:
final_df

,address,timestamp,realized_balance
0,0x0000000000000000000000000000000000000000,2024-01-07 15:25:00+00:00,0.000000
1,0x0000000000000000000000000000000000000000,2024-01-08 17:49:00+00:00,0.000425
2,0x0000000000000000000000000000000000000000,2024-01-25 15:18:00+00:00,0.000425
3,0x0000000000000000000000000000000000000000,2024-01-25 18:02:00+00:00,0.076445
4,0x0000000000000000000000000000000000000000,2024-02-27 14:08:00+00:00,167.777342
...,...,...,...
519599,0xfffb40347816873ca038756cc87bc610144166b2,2024-02-04 10:25:00+00:00,116.564629
519600,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-28 05:20:00+00:00,9577.678563
519601,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-29 02:17:00+00:00,10157.923101
519602,0xfffe980de8f812bf3731d8f0682e21a2177bb40a,2024-01-29 02:17:00+00:00,0.000000


In [86]:
final_df.to_csv('degen_realized_balanace.csv', index = False)

In [87]:
file_path = r'degen_realized_balanace.csv'
with open(file_path) as file:
    data = file.read()

In [88]:
%%time

table = dune.upload_csv(
    data=data,
    table_name="degen_realized_balanace",
    is_private=False
)

In [89]:
table

True

In [91]:
os.path.join(os.path.dirname(__file__), '..', 'uploads')

NameError: name '__file__' is not defined

In [93]:
os.path.join(os.path.abspath(__name__), '..', '.env')

'/Users/zokum/local-Workspace/misc/degen_dash/__main__/../.env'